# DLE303 Assessment 3 - Main
## Rachel Tucker & Michael Cuffe

## Dependencies:
- Python 3.12

In [40]:
%%capture
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

# import libraries

In [41]:
import transformers
from transformers import trainer
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from accelerate.test_utils.scripts.test_distributed_data_loop import test_data_loader
import torch
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd

In [42]:
# Ignore warnings after first viewing
import warnings
warnings.filterwarnings(action='once')

# This is where we split the data into training and testing sets 

In [43]:
# Load the dataset
data = pd.read_csv('data/200k_Short_Texts.csv')
data = data.dropna()
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the Data

In [ ]:
tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

def tokenize_data(data):
    return tokenizer(
        data['text'].tolist(),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt',
    )

train_encodings = tokenize_data(train_data)
test_encodings = tokenize_data(test_data)

C:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Create a Dataset class

In [30]:
class HumorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)


# Flatten\ing (Kind of)

In [31]:
train_labels = train_data['humor'].astype(int).tolist()
test_labels = test_data['humor'].astype(int).tolist()

train_dataset = HumorDataset(train_encodings, train_labels)
test_dataset = HumorDataset(test_encodings, test_labels)

# Define the Model

In [32]:
model = BertForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2);

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Initial Training

In [ ]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    f1 = f1_score(labels, preds, average='weighted')
    return {"f1": f1}

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=600,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=6000,
    eval_strategy="steps",  # Evaluate every `logging_steps`
    eval_steps=6000,  # Evaluation and logging will be done every 100 steps
    save_steps=10000,
    save_total_limit=2,  # Only last 2 models are saved
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

# Fine Tuning the Model

In [33]:
# Use the torch.optim.AdamW optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Use the get_scheduler function to create a linear scheduler
num_training_steps = len(train_dataloader) * 3
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
# Define the loss function
loss_fn = CrossEntropyLoss()

In [ ]:
# Implement the tuning
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

In [ ]:
model.eval()
predictions, true_labels = [], []
for batch in test_data_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions.append(logits)
    true_labels.append(batch['labels'])

In [ ]:
print("Accuracy:", accuracy_score(true_labels, predictions))

## Train the Model

In [37]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=600,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=6000,
    eval_strategy="steps",  # Evaluate every `logging_steps`
    eval_steps=6000,  # Evaluation and logging will be done every 100 steps
    save_steps=10000,
    save_total_limit=2,  # Only last 2 models are saved
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,F1
6000,0.083800,0.095020,0.977098


KeyboardInterrupt: 

end of file